# Searching for files

In [57]:
import logging, os, json

def locate_files(arg):
    directories = ["C:\\Users\\cwest\\Videos\\BoilestDev"]
    extensions = ['.mp4', '.mkv', '.avi']

    print(f'Searching directories: {directories}')
    print(f'File extensions: {extensions}')

    for file_located in find_files(directories, extensions):
        logging.debug('File located, sending to ffprobe function')
        try:
            file_located_data = json.loads(file_located)
            logging.debug(json.dumps(file_located_data, indent=3, sort_keys=True))
            print(json.dumps(file_located_data, indent=3, sort_keys=True))
            # >>>>>>>>>>><<<<<<<<<<<<<<<<
            # >>>>>>>>>>><<<<<<<<<<<<<<<<
            print(file_located_data)
            # >>>>>>>>>>><<<<<<<<<<<<<<<<
            # >>>>>>>>>>><<<<<<<<<<<<<<<<
        except json.JSONDecodeError as e:
            logging.error(f'Failed to decode JSON: {e}')
            continue

def find_files(directories, extensions):
    for directory in directories:
        logging.info ('Scanning: ' + directory)
        for root, dirs, files in os.walk(directory):
            for file in files:
                for ext in extensions:
                    if file.lower().endswith(ext.lower()):
                        file_path = os.path.join(root, file)
                        result_dict = {
                            'directory': directory,
                            'root': root,
                            'file': file,
                            'file_path': file_path,
                            'extension': ext
                        }
                        yield json.dumps(result_dict)

In [58]:
locate_files('farts')

Searching directories: ['C:\\Users\\cwest\\Videos\\BoilestDev']
File extensions: ['.mp4', '.mkv', '.avi']
{
   "directory": "C:\\Users\\cwest\\Videos\\BoilestDev",
   "extension": ".mp4",
   "file": "test - Copy (2).mp4",
   "file_path": "C:\\Users\\cwest\\Videos\\BoilestDev\\test - Copy (2).mp4",
   "root": "C:\\Users\\cwest\\Videos\\BoilestDev"
}
{'directory': 'C:\\Users\\cwest\\Videos\\BoilestDev', 'root': 'C:\\Users\\cwest\\Videos\\BoilestDev', 'file': 'test - Copy (2).mp4', 'file_path': 'C:\\Users\\cwest\\Videos\\BoilestDev\\test - Copy (2).mp4', 'extension': '.mp4'}
{
   "directory": "C:\\Users\\cwest\\Videos\\BoilestDev",
   "extension": ".mp4",
   "file": "test - Copy (3).mp4",
   "file_path": "C:\\Users\\cwest\\Videos\\BoilestDev\\test - Copy (3).mp4",
   "root": "C:\\Users\\cwest\\Videos\\BoilestDev"
}
{'directory': 'C:\\Users\\cwest\\Videos\\BoilestDev', 'root': 'C:\\Users\\cwest\\Videos\\BoilestDev', 'file': 'test - Copy (3).mp4', 'file_path': 'C:\\Users\\cwest\\Videos\\Boi

# Running FFprobe

In [46]:
import subprocess, json, os

def check_video_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the video stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    desired_video_codec = 'av1'
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    if codec_name == desired_video_codec:
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:v copy'
    elif codec_name == 'mjpeg':
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:v copy'
    elif codec_name != desired_video_codec: 
        encoding_decision = True
        svt_av1_string = "libsvtav1 -crf 25 -preset 4 -g 240 -pix_fmt yuv420p10le -svtav1-params filmgrain=20:film-grain-denoise=0:tune=0:enable-qm=1:qm-min=0:qm-max=15"
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:v ' + svt_av1_string
    else:
        print ('ignoring for now')
    return encoding_decision, ffmpeg_command

def check_audio_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the audio stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    # This will be populated at a later date
    #desired_audio_codec = 'aac'
    #if codec_name != desired_video_codec:
    #    encoding_decision = True
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:a copy'
    return encoding_decision, ffmpeg_command
    
def check_subtitle_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the subtitle stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    # This will be populated at a later date
    #desired_subtitle_codec = 'srt'
    #if codec_name != desired_subtitle_codec:
    #    encoding_decision = True
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:s copy'
    return encoding_decision, ffmpeg_command

def check_attachmeent_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the attachment stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    # This will be populated at a later date
    #desired_attachment_codec = '???'
    #if codec_name != desired_attachment_codec:
    #    encoding_decision = True
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:t copy'
    return encoding_decision, ffmpeg_command

def check_codecs(stream_info, encoding_decision):
    # Loops through the streams in stream_info from requires_encoding, then
    # calls functions to determine if the steam needs encoding based on stream type conditions 
    streams_count = stream_info['format']['nb_streams']
    ffmpeg_command = str()
    print ('There are : ' + str(streams_count) + ' streams')
    for i in range (0,streams_count):
        codec_type = stream_info['streams'][i]['codec_type'] 
        if codec_type == 'video':
            encoding_decision, ffmpeg_command = check_video_stream(encoding_decision, i, stream_info, ffmpeg_command)
        elif codec_type == 'audio':
            encoding_decision, ffmpeg_command = check_audio_stream(encoding_decision, i, stream_info, ffmpeg_command)
        elif codec_type == 'subtitle':
            encoding_decision, ffmpeg_command = check_subtitle_stream(encoding_decision, i, stream_info, ffmpeg_command)
        elif codec_type == 'attachment':
            encoding_decision, ffmpeg_command = check_attachmeent_stream(encoding_decision, i, stream_info, ffmpeg_command)        
    return encoding_decision, ffmpeg_command


def check_container_extension(file, encoding_decision):
    base, ext = os.path.splitext(file)
    if ext.lower() != '.mkv':
        # Change the extension to .mkv
        file = base + '.mkv'
        encoding_decision = True
    ffmepg_output_file = '/boil_hold/' + file
    return encoding_decision, ffmepg_output_file

def check_container_type(stream_info, encoding_decision, file):
    # Desired container is MKV so we check for that, and pass True for all other container types
    format_name = stream_info['format'].get('format_name')
    print ('format is: ' + format_name)
    if format_name != 'matroska,webm':
        encoding_decision = True
    encoding_decision, ffmepg_output_file = check_container_extension(file, encoding_decision)
    print ('>>>check_container_type<<<  Container is: ' + format_name + ' so, encoding_decision is: ' + str(encoding_decision))
    return encoding_decision, ffmepg_output_file
    

def ffprobe_function(file_path):
    # Subprocess call to ffprobe to retrieve video info in JSON format
    ffprobe_command = f'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout,format=nb_streams -of json "{file_path}"'
    result = subprocess.run(ffprobe_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stream_info = json.loads(result.stdout)
    return stream_info

def file_size_kb(file_path):
    # Returns the file size of the file_path on disk
    if os.path.isfile(file_path):
        file_size_bytes = os.path.getsize(file_path)
        file_size_kb = file_size_bytes / 1024
        return round(file_size_kb)
    else:
        return 0.0
    

def requires_encoding(file_located_data):
    stream_info = ffprobe_function(file_located_data['file_path'])
    encoding_decision = False
    
    encoding_decision, ffmepg_output_file_name = check_container_type(stream_info, encoding_decision, file_located_data['file'])
    encoding_decision, ffmpeg_command = check_codecs(stream_info, encoding_decision)
    if encoding_decision == True:
        print ('file needs encoding')
        file_located_data['ffmpeg_command'] = ffmpeg_command
        file_located_data['ffmepg_output_file_name'] = ffmepg_output_file_name
        file_located_data['file_hash'] = file_size_kb(file_located_data['file_path'])
        print(json.dumps(file_located_data, indent=4))
    else:
        print ('file does not need encoding')
    print (encoding_decision)
    print (ffmpeg_command)


In [59]:
file_located_data = {
   "directory": "C:\\Users\\cwest\\Videos\\BoilestDev",
   "extension": ".mkv",
   "file": "test1.mkv",
   "file_path": "C:\\Users\\cwest\\Videos\\BoilestDev\\test1.mkv",
   "root": "C:\\Users\\cwest\\Videos\\BoilestDev"
}
requires_encoding(file_located_data)

format is: matroska,webm
>>>check_container_type<<<  Container is: matroska,webm so, encoding_decision is: False
There are : 10 streams
Steam 0 codec is: hevc
Steam 1 codec is: aac
Steam 2 codec is: aac
Steam 3 codec is: ass
Steam 4 codec is: ass
Steam 5 codec is: ttf
Steam 6 codec is: otf
Steam 7 codec is: otf
Steam 8 codec is: otf
Steam 9 codec is: otf
file needs encoding
{
    "directory": "C:\\Users\\cwest\\Videos\\BoilestDev",
    "extension": ".mkv",
    "file": "test1.mkv",
    "file_path": "C:\\Users\\cwest\\Videos\\BoilestDev\\test1.mkv",
    "root": "C:\\Users\\cwest\\Videos\\BoilestDev",
    "ffmpeg_command": " -map 0:0 -c:v libsvtav1 -crf 25 -preset 4 -g 240 -pix_fmt yuv420p10le -svtav1-params filmgrain=20:film-grain-denoise=0:tune=0:enable-qm=1:qm-min=0:qm-max=15 -map 0:1 -c:a copy -map 0:2 -c:a copy -map 0:3 -c:s copy -map 0:4 -c:s copy -map 0:5 -c:t copy -map 0:6 -c:t copy -map 0:7 -c:t copy -map 0:8 -c:t copy -map 0:9 -c:t copy",
    "ffmepg_output_file_name": "/boil_ho

# Running FFMpeg

In [82]:
import subprocess, json, os, shutil

def process_ffmpeg(file_located_data):
    file_located_data['ffmpeg_prelaunch_checks'] = False
    file_located_data['run_ffmpeg'] = False
    file_located_data['ffmpeg_postlaunch_checks'] = False
    file_located_data['move_media'] = False

    print('Running process_ffmpeg workflow')

    file_located_data = ffmpeg_prelaunch_checks(file_located_data)

    print ('Outcome of ffmpeg_prelaunch_checks is: ' + str(file_located_data['ffmpeg_prelaunch_checks']))

    if file_located_data['ffmpeg_prelaunch_checks'] == True:
        file_located_data = run_ffmpeg(file_located_data)
    
    print('Outcome of run_ffmpeg is: ' + str(file_located_data['run_ffmpeg']))

    if file_located_data['run_ffmpeg'] == True:
        file_located_data = ffmpeg_postlaunch_checks(file_located_data)

    print('Outcome of ffmpeg_postlaunch_checks is: ' + str(file_located_data['ffmpeg_postlaunch_checks']))

    if file_located_data['ffmpeg_postlaunch_checks'] == True:
        file_located_data = move_media(file_located_data)
    
    print('Outcome of move_media is: ' + str(file_located_data['move_media']))

    if file_located_data['move_media'] == True:
        ###################
        ###################
        ###################
        print('calling the function to write to DB will go here')
        ###################
        ###################
        ###################
    
    print('done done done workflow')


################# Pre Launch Checks #################

def ffmpeg_prelaunch_checks(file_located_data):
    file_located_data['prelaunch_file_exists'] = False
    file_located_data['prelaunch_hash_match'] = False
    file_located_data['prelaunch_file_validation'] = False

    file_located_data = prelaunch_file_exists(file_located_data)

    if file_located_data['prelaunch_file_exists'] == True:
        file_located_data = prelaunch_hash_match(file_located_data)
    
    if file_located_data['prelaunch_hash_match'] == True:
        file_located_data = prelaunch_file_validation(file_located_data)

    if file_located_data['prelaunch_file_exists'] == True and file_located_data['prelaunch_hash_match'] == True and file_located_data['prelaunch_file_validation'] == True:
        file_located_data['ffmpeg_prelaunch_checks'] = True

    return file_located_data


def prelaunch_file_exists(file_located_data):
    #  Checks to see if the input file still exists, returns True on existance
    if file_exists(file_located_data['file_path']) == True:
        print('File exists')
        file_located_data['prelaunch_file_exists'] = True
    print('Outcome of prelaunch_file_exists is: ' + str(file_located_data['prelaunch_file_exists']))
    return file_located_data


def prelaunch_hash_match(file_located_data):
    original_file_hash = file_located_data['file_hash'] 
    print('Original file hash is: ' + str(original_file_hash))
    current_file_hash = file_size_kb(file_located_data['file_path'])
    print('Current file hash is: ' + str(current_file_hash))
    if original_file_hash == current_file_hash:
        print('Hashes match')
        file_located_data['prelaunch_hash_match'] = True
    else:
        print('Hashes do not match')
    print('Outcome of prelaunch_hash_match is: ' + str(file_located_data['prelaunch_hash_match']))
    return file_located_data


def prelaunch_file_validation(file_located_data):
    filepath = file_located_data['file_path']
    if validate_video(filepath) == True:
        print('Input file passed validation')
        file_located_data['prelaunch_file_validation'] = True
    else:
        print('Input file failed validation')
    print('Outcome of prelaunch_file_validation is: ' + str(file_located_data['prelaunch_file_validation']))
    return file_located_data    


################# Run FFMPEG  #################

def run_ffmpeg(file_located_data):
    file_located_data['ffmpeg_string_ready'] = False
    file_located_data['video_encoded'] = False
    
    file_located_data = ffmpeg_string_ready(file_located_data)

    if file_located_data['ffmpeg_string_ready'] == True:
        file_located_data = video_encoded(file_located_data)
    
    if file_located_data['ffmpeg_string_ready'] == True and file_located_data['video_encoded'] == True:
        file_located_data['run_ffmpeg'] = True

    return file_located_data
    
    
def ffmpeg_string_ready(file_located_data):
    full_ffmpeg_command = str()
    ffmpeg_settings = 'ffmpeg -hide_banner -loglevel 16 -stats -stats_period 10'
    file_path = file_located_data['file_path']
    ffmpeg_command = file_located_data['ffmpeg_command']
    ffmepg_output_file_name = file_located_data['ffmepg_output_file_name']

    full_ffmpeg_command = ffmpeg_settings + ' -y -i "' + file_path + '" ' + ffmpeg_command + ' "' + ffmepg_output_file_name + '"'
    print ('ffmpeg_command is: ' + full_ffmpeg_command)

    # Someday, add some kind of validation to the ffmpeg command

    file_located_data['full_ffmpeg_command'] = full_ffmpeg_command
    file_located_data['ffmpeg_string_ready'] = True

    print('Outcome of ffmpeg_string_ready is: ' + str(file_located_data['ffmpeg_string_ready']))
    return file_located_data
    

def video_encoded(file_located_data):
    full_ffmpeg_command = file_located_data['full_ffmpeg_command']
    if run_ffmpeg(full_ffmpeg_command) == True:
        file_located_data['video_encoded'] = True 
    print('Outcome of video_encoded is: ' + str(file_located_data['video_encoded']))
    return file_located_data



################# Post Launch Checks #################


def ffmpeg_postlaunch_checks(file_located_data):
    file_located_data['post_launch_file_check'] = False
    file_located_data['post_launch_file_size_check'] = False
    file_located_data['post_launch_file_validation'] = False
    
    file_located_data = post_launch_file_check(file_located_data)

    if file_located_data['post_launch_file_check'] == True:
        file_located_data = post_launch_file_size_check(file_located_data)

    if file_located_data['post_launch_file_size_check'] == True:
        file_located_data = post_launch_file_validation(file_located_data)
    
    if file_located_data['post_launch_file_validation'] == True:
        file_located_data['ffmpeg_postlaunch_checks'] = True        

    return file_located_data


def post_launch_file_check(file_located_data):
    # Check to see if the original file, and the encoded file are there
    original_file = file_located_data['file_path']
    encoded_file = file_located_data['ffmepg_output_file_name']
    
    if file_exists(original_file) == True and file_exists(encoded_file) == True:
        file_located_data['post_launch_file_check'] = True

    print('Outcome of post_launch_file_check is: ' + str(file_located_data['post_launch_file_check']))
    return file_located_data


def post_launch_file_size_check(file_located_data):
    # Some ffmpeg errors result in a file size of 0KB.  This checks for that.
    # Also attaches the file size of the encoded output
    encoded_file_size = get_file_size_kb(file_located_data['file_path'])
    if encoded_file_size > 0.0:
        file_located_data['encoded_file_size'] = encoded_file_size
        file_located_data['post_launch_file_size_check'] = True
    print('Outcome of post_launch_file_size_check is: ' + str(file_located_data['post_launch_file_size_check']))
    return file_located_data


def post_launch_file_validation(file_located_data):
    filepath = file_located_data['ffmepg_output_file_name']
    if validate_video(filepath) == True:
        file_located_data['post_launch_file_validation'] = True
    else:
        print('Output file failed validation')
    print('Outcome of post_launch_file_validation is: ' + str(file_located_data['post_launch_file_validation']))
    return file_located_data    


################# move_media #################


def move_media(file_located_data):
    # All of the related operations to move the encoded file to the correct directory + cleanup opperations
    file_located_data['rename_original_file'] = False
    file_located_data['move_encoded_file'] = False
    file_located_data['delete_renamed_original_file'] = False

    file_located_data = rename_original_file(file_located_data) 

    if file_located_data['rename_original_file'] == True:
        file_located_data = move_encoded_file(file_located_data) 
    
    print('Outcome of move_media is: ' + str(file_located_data['move_media']))
    return file_located_data


def rename_original_file(file_located_data):
    # This is here incase any of the move opperations mess up
    file_path = file_located_data['file_path']
    directory, filename = os.path.split(file_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}-copy{ext}"
    new_file_path = os.path.join(directory, new_filename)
    os.rename(file_path, new_file_path)

    if file_exists(new_file_path):
        file_located_data['rename_original_file'] = True
        file_located_data['new_file_path'] = new_file_path
    
    print('Outcome of rename_original_file is: ' + str(file_located_data['rename_original_file']))
    return file_located_data


def move_encoded_file(file_located_data):
    # Function to move the encoded file to the original file's directory
    file_path = file_located_data['file_path']
    ffmepg_output_file_name = file_located_data['ffmepg_output_file_name']
    new_file_destination = destination_file_name(file_path, ffmepg_output_file_name)
    print('new_file_destination: ' + str(new_file_destination))
    try:
        print('removing: ' + str(file_path))
        os.remove(file_path)
        print('moving: ' + str(ffmepg_output_file_name) + ' to: ' + str(new_file_destination))
        shutil.move(ffmepg_output_file_name, new_file_destination) 
        file_located_data['move_processed_file'] = True
    except Exception as e:
        print(f"An error occurred: {e}")
    
    if file_exists(new_file_destination):
        file_located_data['move_encoded_file'] = True
    
    return file_located_data


def destination_file_name (file_path, ffmepg_output_file_name):
    # Quick and silly function for creating the correct filepath to move the encoded file to
    destination_file_name = os.path.join(os.path.dirname(file_path), os.path.basename(ffmepg_output_file_name))
    return destination_file_name


def delete_renamed_original_file(file_located_data):
    # Function to delete the renamed original file
    new_file_path = file_located_data['new_file_path']
    os.remove(new_file_path)

    if file_exists(new_file_path):
        file_located_data['delete_renamed_original_file'] = True
    print('Outcome of delete_renamed_original_file is: ' + str(file_located_data['delete_renamed_original_file']))
    return file_located_data

########################## Common Functions ##########################

def file_exists(filepath):
    file_existance = os.path.isfile(filepath)
    # Returns true if the file that is about to be touched is in the expected location
    print (filepath + ' : ' + str(file_existance))
    return file_existance

def get_file_size_kb(filepath):
    file_size_bytes = os.path.getsize(filepath)
    file_size_kb = round(file_size_bytes / 1024)
    return file_size_kb

def validate_video(filepath):
    # This function determines if a video is valid, or if the video contains errors
    # Returns:
    #       Failure if the shell command returns anything; i.e. one of the streams is bad
    #       Success if the shell command doesn't return anything; i.e. the streams are good
    #       Error if the shell command fails; this shouldn't happen
    try:
        command = 'ffmpeg -v error -i "' + filepath + '" -f null -'
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.stdout or result.stderr:
            print ('File failed validation')
            return False
        else:
            print ('File passed validation')
            return True
    except Exception as e:
        return f"Error: {e}"
    

def run_ffmpeg(ffmpeg_command):
    # Command to run ffmpeg in subprocess
    print ('running ffmpeg now')
    try:
        process = subprocess.Popen(ffmpeg_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True)
        for line in process.stdout:
            print(line)
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False  # Return a non-zero exit code to indicate an error


In [83]:
file_located_data= {
    "directory": "C:\\Users\\cwest\\Videos\\BoilestDev",
    "extension": ".mkv",
    "file": "test1.mkv",
    "file_path": "C:\\Users\\cwest\\Videos\\BoilestDev\\test1.mkv",
    "root": "C:\\Users\\cwest\\Videos\\BoilestDev",
    "ffmpeg_command": " -map 0:0 -c:v libsvtav1 -crf 25 -preset 4 -g 240 -pix_fmt yuv420p10le -svtav1-params filmgrain=20:film-grain-denoise=0:tune=0:enable-qm=1:qm-min=0:qm-max=15 -map 0:1 -c:a copy -map 0:2 -c:a copy -map 0:3 -c:s copy -map 0:4 -c:s copy -map 0:5 -c:t copy -map 0:6 -c:t copy -map 0:7 -c:t copy -map 0:8 -c:t copy -map 0:9 -c:t copy",
    "ffmepg_output_file_name": "/boil_hold/test1.mkv",
    "file_hash": 361932
}

#ffmpeg_prelaunch_checks(file_located_data)
#ffmpeg_postlaunch_checks(file_located_data)
process_ffmpeg(file_located_data)
#process_file(file_located_data)
#validate_video(file_located_data['file_path'])
#validate_video(file_located_data['ffmepg_output_file_name'])
print('manual test done')

Running process_ffmpeg workflow
C:\Users\cwest\Videos\BoilestDev\test1.mkv : True
File exists
Outcome of prelaunch_file_exists is: True
Original file hash is: 361932
Current file hash is: 361932
Hashes match
Outcome of prelaunch_hash_match is: True
File passed validation
Input file passed validation
Outcome of prelaunch_file_validation is: True
Outcome of ffmpeg_prelaunch_checks is: True
running ffmpeg now
'directory' is not recognized as an internal or external command,

operable program or batch file.



TypeError: 'bool' object is not subscriptable

----
# Writing to DB
----

In [5]:
import mysql.connector
from mysql.connector import Error
from datetime import datetime

def insert_record(unique_identifier, file_name, file_path, config_name, new_file_size, new_file_size_difference, old_file_size, watch_folder, ffmpeg_encoding_string):
    try:
        # Connection details
        connection = mysql.connector.connect(
            host='192.168.1.110',
            port=32053,  # replace with your non-default port
            database='boilest',
            user='boilest',
            password='boilest'
        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            recorded_date = datetime.now()  # Current date and time
            
            insert_query = """
                INSERT INTO ffmpeghistory (
                    unique_identifier, recorded_date, file_name, file_path, config_name,
                    new_file_size, new_file_size_difference, old_file_size, watch_folder, ffmpeg_encoding_string
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            record = (
                unique_identifier, recorded_date, file_name, file_path, config_name,
                new_file_size, new_file_size_difference, old_file_size, watch_folder, ffmpeg_encoding_string
            )
            
            cursor.execute(insert_query, record)
            connection.commit()
            print("Record inserted successfully")
            
    except Error as e:
        print(f"Error while connecting to MariaDB: {e}")
    
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MariaDB connection is closed")



In [6]:
# Example usage
insert_record(
    unique_identifier='12345',
    file_name='example.mp4',
    file_path='/path/to/example.mp4',
    config_name='default_config',
    new_file_size=1024,
    new_file_size_difference=100,
    old_file_size=924,
    watch_folder='/path/to/watch',
    ffmpeg_encoding_string='ffmpeg -i input.mp4 output.mp4'
)


Record inserted successfully
MariaDB connection is closed
